In [26]:
from funct import *

In [27]:
from langchain_community.utilities import SQLDatabase

# 🔹 Step 1: Connect to SQLite Database
db = SQLDatabase.from_uri("sqlite:///Word_DB.sqlite")

# 🔹 Step 2: Run the Query to Get Words
query = "SELECT word FROM words WHERE fluency = 5;"
result = db.run(query)

# 🔹 Step 3: Print Results
print("Words with fluency = 5:", result)


Words with fluency = 5: [(' tout',), (' chat',), (' silence',), (' fruit',)]


In [33]:
import ollama

client = ollama.Client()

model="gemma3:4b" 

def sentence_gen(word):
    '''creates a simple example sentnece using the word provided as well as words from ones libary based on your fluceny level of them
    - sort by word only above fluency level 3.    or something. 
    '''

    # frist attempt is basic and will just return any sentence with word in it.
    prompt = f"Only return a very basic example of a French sentence that includes this french word: '{word}'. if possible try to use a word from this list that follows but ONLY if it makes sense else do not use any words in list: {result}"

    return (client.generate(model=model, prompt=prompt).response)#.split("</think>")[1][2:]

In [29]:
#exiting fluceny here
from langchain_community.utilities import SQLDatabase

# 🔹 Step 1: Connect to SQLite Database
db = SQLDatabase.from_uri("sqlite:///Word_DB.sqlite")

# 🔹 Step 2: Define the Update Query
update_query = "UPDATE words SET fluency = 5 WHERE id = 177;"

# 🔹 Step 3: Execute the Query
db.run(update_query)

print("Fluency updated for ID.")


Fluency updated for ID.


In [30]:


# print(sentence_gen("et"))
# print(sentence_gen("cause"))
# print(sentence_gen("chat"))
# print(sentence_gen("papa"))
# print(sentence_gen("que"))
# print(sentence_gen("souvenir"))



In [31]:
def give_me_a_word():
    '''Will open up a window and will give the next practice word for you to learn or to review pygame pop up? -- should call from most and 
    using the gen sentece pornounce and more
    '''

    return 0


def pick_next_word():
    '''will use an algo to start picking next words ect..'''

    return 0

# this needs to be in this directly else it won't work on newly added mp3
def mp3_play(word):
    mp3_path = f"Pronunciations\{word}.mp3"

    if not os.path.exists(mp3_path):  
       print("MP3 file not found.") 
       return
    
    pygame.mixer.init()
    pygame.mixer.music.load(f"Pronunciations\{word}.mp3")
    pygame.mixer.music.play()
    
    print(f"Playing: {word}")

    # this might be required later so i'll keep it open like this
    while pygame.mixer.music.get_busy():  # Keep script running while music plays
        continue
    else:
        pygame.mixer.quit()  # closes it so less of a problem for other functions


In [34]:
import pygame

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 500, 700
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Pygame Lang-app V0.1")

# Colors
WHITE = (255, 255, 255)
GRAY = (200, 200, 200)
DARK_GRAY = (150, 150, 150)
BLUE = (0, 122, 204)
BLACK = (0, 0, 0)

# Font
font = pygame.font.Font(None, 50)
button_font = pygame.font.Font(None, 30)

# Game state
current_screen = "main"
 
# Button properties
button1_rect = pygame.Rect(100, 350, 300, 50)
button2_rect = pygame.Rect(100, 420, 300, 50)
button3_rect = pygame.Rect(100, 490, 300, 50)
function_button_rect = pygame.Rect(50, 200, 120, 50)
print_button_rect = pygame.Rect(50, 500, 120, 50)
next_w_button_rect = pygame.Rect(300, 500, 120, 50)
p_button_word_react = pygame.Rect(50, 270, 120, 50)
p_button_sent_react = pygame.Rect(300, 270, 120, 50)

def draw_main_screen():
    screen.fill(WHITE)
    title_text = font.render("Mercurius Demo ", True, BLACK)
    screen.blit(title_text, (110, 100))
    
    pygame.draw.rect(screen, BLUE, button1_rect)
    pygame.draw.rect(screen, BLUE, button2_rect)
    pygame.draw.rect(screen, BLUE, button3_rect)
    
    screen.blit(button_font.render("Study", True, WHITE), button1_rect.move(50, 15))
    screen.blit(button_font.render("DO **NOT** CLICK HERE", True, WHITE), button2_rect.move(20, 15))
    screen.blit(button_font.render("Complete Phrases", True, WHITE), button3_rect.move(50, 15))


word_index = 0
val = 0 

def get_current_word():
    #global word_index
    return words[word_index]

function_output = ""

def function_action():
    global function_output
    function_output = sentence_gen(words[word_index].lower())

def card_display():
    word = get_current_word()
    global val
    if val == 0:
        return word
    else: 
        return word + " | " + generate_def(word)

def next_word():
    global word_index
    global function_output
    word_index = (word_index + 1) % len(words)
    function_output = ""



def draw_phrases_screen():
    pass 

def draw_review_screen():

    screen.fill(WHITE)
    word_box_rect = pygame.Rect(100, 50, 300, 60)
    pygame.draw.rect(screen, DARK_GRAY, word_box_rect, border_radius=10)


  # Draw the word in the center of the box
    text_surface = font.render(card_display(), True, BLACK)
    text_rect = text_surface.get_rect(center=word_box_rect.center)
    screen.blit(text_surface, text_rect)

    
    
    pygame.draw.rect(screen, BLUE, function_button_rect)
    pygame.draw.rect(screen, BLUE, print_button_rect)
    pygame.draw.rect(screen, BLUE, next_w_button_rect)
    pygame.draw.rect(screen, BLUE, p_button_word_react)
    pygame.draw.rect(screen, BLUE, p_button_sent_react)
    
    screen.blit(button_font.render("Example", True, WHITE), function_button_rect.move(20, 15))
    screen.blit(button_font.render("English", True, WHITE), print_button_rect.move(35, 15))
    screen.blit(button_font.render("Next", True, WHITE), next_w_button_rect.move(35, 15))
    screen.blit(button_font.render("P_Word", True, WHITE), p_button_word_react.move(20, 15))
    screen.blit(button_font.render("P_Sentence", True, WHITE), p_button_sent_react.move(20, 15))

    output_surface = button_font.render(function_output.split(".")[0], True, BLACK)
    screen.blit(output_surface, (180, 215))




def main():
    global current_screen
    global val
    running = True
    
    while running:
        screen.fill(WHITE)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
            elif event.type == pygame.MOUSEBUTTONDOWN:
                if current_screen == "main":
                    if button1_rect.collidepoint(event.pos):
                        current_screen = "review"
                    elif button2_rect.collidepoint(event.pos): # the "Do not press button"
                        pass  
                    elif button3_rect.collidepoint(event.pos):
                        current_screen = "phrases"
                if current_screen == "review":
                    if function_button_rect.collidepoint(event.pos):
                        function_action()
                    elif p_button_word_react.collidepoint(event.pos):
                        mp3_play(get_current_word())
                    elif p_button_sent_react.collidepoint(event.pos):
                        create_pronunciation(function_output.split(".")[0])
                        mp3_play(function_output.split(".")[0])
                    elif print_button_rect.collidepoint(event.pos):
                        val = 1
                    elif next_w_button_rect.collidepoint(event.pos):
                        val = 0
                        next_word()

                        
                
        if current_screen == "main":
            draw_main_screen()
        elif current_screen == "review":
            draw_review_screen()
        elif current_screen == "phrases":
            draw_phrases_screen()
        
        pygame.display.flip()
    
    pygame.quit()

if __name__ == "__main__":
    main()